In [0]:
import dlt
import pyspark.sql.functions as sf

In [0]:
@dlt.table(
    comment = "Ingesting data from the simulated append-only stream source data",
    name = "nyctaxidata_ingested_from_trips_src",
    temporary = False
)

def ingest_from_streaming_src():

    trips_df = (
        spark.readStream
        .format("delta")
        .table("nyctaxi_copy.trips_src")
    )
    
    enhanced_trips_df = (
        trips_df
        .withColumn(
            "trip_duration_mins",
            sf.round((sf.col("tpep_dropoff_datetime").cast("long") 
            - sf.col("tpep_pickup_datetime").cast("long")) / 60)
        )
        .withColumn(
            "ingestion_timestamp",
            sf.current_timestamp()
        )
    )
    
    return enhanced_trips_df
